In [ ]:
import os
import sys

local_path = os.getcwd()
# 将项目主目录路径添加到 Python 路径
os.chdir("../../")  # 使用相对路径将工作目录切换到 project 文件夹
project_path = os.path.abspath(os.path.join(local_path, "../../"))
sys.path.append(project_path)   #将模块查找路径切换

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from models import modelset
from train.train import train_FBM
from train.train import DFBM
from utils import *

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
set_seed(42)

In [ ]:
class_counts = [100]*10
datatype = 'KMNIST'

train_loader = get_dataloader(datatype, batch_size=64, train=True, class_counts=class_counts)
test_loader = get_dataloader(datatype, batch_size=64, train=False)

data_iter = iter(train_loader)
images, labels = next(data_iter)
batch, channel, large, _ = images.shape

In [ ]:
import optuna
import csv
csv_file_path = os.path.join(local_path, 'optuna_results.csv')

# 如果文件不存在，则创建文件并写入标题行
if not os.path.exists(csv_file_path):
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['trial_number', 'output_size_list', 'd_f_list', 'alpha_list', 'accuracy'])

# 逐层训练的FBM

In [ ]:
'''
    进行一次推理的过程
'''

# 定义超参数
images_size = channel * large**2 # MNIST图像大小是28x28
num_classes = 10      # MNIST有10个类别
learning_rate = 0.01
lam = 0.01
num_epochs = 20
batch_size = 64

output_size_list = [1000, 1000, 1000, 1000, 1000]
d_f_list = [0.0, 0.04, 0.1, 0.3, 0.5]
alpha_list = [0.0, 100, 5.0, 2.0, 1.0]
deep = 1

pipeline, deal_train_loader =DFBM(train_loader, output_size_list, d_f_list, alpha_list, device)

output_size = output_size_list[-1]

# 定义损失函数和优化器
model2 = modelset.MLP(output_size, num_classes).to(device)
criterion2 = nn.CrossEntropyLoss()  # 使用交叉熵损失
optimizer = optim.Adam(model2.parameters(), lr=learning_rate)  # 使用随机梯度下降优化器

model2.train()
# 训练模型
epochs = 30
for epoch in range(epochs):
    for images, labels in deal_train_loader:
        # 将图像展平为一维向量，并将标签进行 one-hot 编码
        images = images.to(device)
        labels_one_hot = F.one_hot(labels, num_classes=num_classes).float().to(device)  # 将标签转换为 one-hot 编码

        # 前向传播
        outputs = model2(images)

        # 计算损失
        loss = criterion2(outputs, labels_one_hot)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    #print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


# 设置模型为评估模式
pipeline.add_model(model2)
pipeline.eval()

# 准确率计数
correct = 0
total = 0

# 禁用梯度计算，加速测试过程
with torch.no_grad():
    for images, labels in test_loader:
        # 将数据加载到 GPU
        images = images.view(-1, images_size).to(device)
        labels = labels.to(device)

        # 前向传播
        outputs = pipeline(images)
        
        # 获取预测结果
        _, predicted = torch.max(outputs, 1)
        
        # 更新计数
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 计算准确率
accuracy = 100 * correct / total
print(f'Accuracy on the test dataset: {accuracy:.2f}%')
#return accuracy

In [ ]:
# 设置函数用于搜索

def objective(trial):
    # 定义超参数
    images_size = channel * large**2 # MNIST图像大小是28x28
    num_classes = 10      # MNIST有10个类别
    learning_rate = 0.01
    lam = 0.01
    num_epochs = 20
    batch_size = 64

    # 使用 suggest_int 和 suggest_float 创建参数数组
    output_size_list = [trial.suggest_int(f'output_size_{i}', 500, 2000) for i in range(3)] 
    d_f_list = [trial.suggest_float(f'd_f_{i}', 0.0, 1.0) for i in range(3)]
    alpha_list = [trial.suggest_float(f'alpha_{i}', 0.1, 2.0) for i in range(3)]

    pipeline, deal_train_loader =DFBM(train_loader, output_size_list, d_f_list, alpha_list, device)

    output_size = output_size_list[-1]

    # 定义损失函数和优化器
    model2 = modelset.MLP(output_size, num_classes).to(device)
    criterion2 = nn.CrossEntropyLoss()  # 使用交叉熵损失
    optimizer = optim.Adam(model2.parameters(), lr=learning_rate)  # 使用随机梯度下降优化器

    model2.train()
    # 训练模型
    epochs = 30
    for epoch in range(epochs):
        for images, labels in deal_train_loader:
            # 将图像展平为一维向量，并将标签进行 one-hot 编码
            images = images.to(device)
            labels_one_hot = F.one_hot(labels, num_classes=num_classes).float().to(device)  # 将标签转换为 one-hot 编码

            # 前向传播
            outputs = model2(images)

            # 计算损失
            loss = criterion2(outputs, labels_one_hot)

            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # 设置模型为评估模式
    pipeline.add_model(model2)
    pipeline.eval()

    # 准确率计数
    correct = 0
    total = 0

    # 禁用梯度计算，加速测试过程
    with torch.no_grad():
        for images, labels in test_loader:
            # 将数据加载到 GPU
            images = images.view(-1, images_size).to(device)
            labels = labels.to(device)

            # 前向传播
            outputs = pipeline(images)
            
            # 获取预测结果
            _, predicted = torch.max(outputs, 1)
            
            # 更新计数
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # 计算准确率
    accuracy = 1.0 * correct / total

    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([trial.number, output_size_list, d_f_list, alpha_list, accuracy])


    return 1 - accuracy

In [ ]:
import optuna

# 创建 Optuna study 并优化
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=3)  # 进行 100 次优化搜索

# 输出最佳参数
print("Best parameters:", study.best_params)
print("Best validation loss:", study.best_value)

# 整体训练FBM

In [ ]:
'''
    进行一次推理的过程
'''
from train.train import pre_DFBM
# 定义超参数
images_size = channel * large**2 # MNIST图像大小是28x28
num_classes = 10      # MNIST有10个类别
learning_rate = 0.01
lam = 0.01
num_epochs = 20
batch_size = 64

output_size_list = [1000, 1000, 1000, 1000, 1000]
d_f_list = [0.0, 0.04, 0.1, 0.3, 0.5]
alpha_list = [0.0, 100, 5.0, 2.0, 1.0]
deep = 1

pipeline, deal_train_loader =pre_DFBM(train_loader, output_size_list, d_f_list, alpha_list, device, train_MLP=True)

output_size = output_size_list[-1]

pipeline.eval()

# 准确率计数
correct = 0
total = 0

# 禁用梯度计算，加速测试过程
with torch.no_grad():
    for images, labels in test_loader:
        # 将数据加载到 GPU
        images = images.view(-1, images_size).to(device)
        labels = labels.to(device)

        # 前向传播
        outputs = pipeline(images)
        
        # 获取预测结果
        _, predicted = torch.max(outputs, 1)
        
        # 更新计数
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 计算准确率
accuracy = 100 * correct / total
print(f'Accuracy on the test dataset: {accuracy:.2f}%')
#return accuracy

In [13]:
# 设置函数用于搜索

def objective(trial):
    # 定义超参数
    images_size = channel * large**2 # MNIST图像大小是28x28
    num_classes = 10      # MNIST有10个类别
    learning_rate = 0.01
    lam = 0.01
    num_epochs = 20
    batch_size = 64

    # 使用 suggest_int 和 suggest_float 创建参数数组
    output_size_list = [trial.suggest_int(f'output_size_{i}', 500, 2000) for i in range(3)] 
    d_f_list = [trial.suggest_float(f'd_f_{i}', 0.0, 1.0) for i in range(3)]
    alpha_list = [trial.suggest_float(f'alpha_{i}', 0.1, 2.0) for i in range(3)]

    pipeline, deal_train_loader = pre_DFBM(train_loader, output_size_list, d_f_list, alpha_list, device, train_MLP=True)

    # 设置模型为评估模式
    pipeline.eval()

    # 准确率计数
    correct = 0
    total = 0

    # 禁用梯度计算，加速测试过程
    with torch.no_grad():
        for images, labels in test_loader:
            # 将数据加载到 GPU
            images = images.view(-1, images_size).to(device)
            labels = labels.to(device)

            # 前向传播
            outputs = pipeline(images)
            
            # 获取预测结果
            _, predicted = torch.max(outputs, 1)
            
            # 更新计数
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # 计算准确率
    accuracy = 1.0 * correct / total

    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([trial.number, output_size_list, d_f_list, alpha_list, accuracy])


    return 1 - accuracy

In [14]:
import optuna

# 创建 Optuna study 并优化
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=3)  # 进行 100 次优化搜索

# 输出最佳参数
print("Best parameters:", study.best_params)
print("Best validation loss:", study.best_value)

[I 2024-11-15 01:04:22,245] A new study created in memory with name: no-name-cf90de25-08d7-4474-90c6-908804983302


torch.Size([64, 1, 28, 28])
Epoch [1/30], Loss: ['0.2272', '0.3214', '0.3457', '2.6200']
Epoch [2/30], Loss: ['0.2229', '0.2810', '0.3201', '1.6832']
Epoch [3/30], Loss: ['0.2215', '0.2744', '0.3019', '1.5062']
Epoch [4/30], Loss: ['0.1989', '0.2380', '0.2313', '1.3279']
Epoch [5/30], Loss: ['0.2006', '0.2352', '0.2301', '1.6601']
Epoch [6/30], Loss: ['0.2038', '0.2447', '0.2495', '1.3525']
Epoch [7/30], Loss: ['0.1997', '0.2394', '0.2375', '0.9479']
Epoch [8/30], Loss: ['0.1971', '0.2336', '0.2342', '1.1970']
Epoch [9/30], Loss: ['0.2064', '0.2362', '0.2387', '1.2642']
Epoch [10/30], Loss: ['0.1745', '0.2136', '0.2035', '0.8374']
Epoch [11/30], Loss: ['0.1965', '0.2335', '0.2250', '0.8966']
Epoch [12/30], Loss: ['0.1869', '0.2309', '0.2144', '0.7979']
Epoch [13/30], Loss: ['0.1860', '0.2219', '0.2126', '0.9876']
Epoch [14/30], Loss: ['0.1884', '0.2218', '0.2151', '1.1715']
Epoch [15/30], Loss: ['0.1855', '0.2258', '0.2179', '1.2083']
Epoch [16/30], Loss: ['0.1993', '0.2363', '0.2175',

[I 2024-11-15 01:04:33,118] Trial 0 finished with value: 0.4679 and parameters: {'output_size_0': 680, 'output_size_1': 1667, 'output_size_2': 546, 'd_f_0': 0.13905803011331241, 'd_f_1': 0.7427427574543171, 'd_f_2': 0.5994859049072435, 'alpha_0': 1.2801852868072683, 'alpha_1': 0.325585086243613, 'alpha_2': 0.9831804569199781}. Best is trial 0 with value: 0.4679.


torch.Size([64, 1, 28, 28])
Epoch [1/30], Loss: ['0.0720', '0.0789', '0.0894', '1.7814']
Epoch [2/30], Loss: ['0.0725', '0.0597', '0.0594', '0.7251']
Epoch [3/30], Loss: ['0.0793', '0.0656', '0.0692', '0.9435']
Epoch [4/30], Loss: ['0.0753', '0.0549', '0.0547', '1.6481']
Epoch [5/30], Loss: ['0.0708', '0.0575', '0.0559', '0.8399']
Epoch [6/30], Loss: ['0.0729', '0.0546', '0.0586', '1.0720']
Epoch [7/30], Loss: ['0.0718', '0.0538', '0.0588', '0.7437']
Epoch [8/30], Loss: ['0.0614', '0.0466', '0.0457', '0.3612']
Epoch [9/30], Loss: ['0.0610', '0.0476', '0.0494', '0.5034']
Epoch [10/30], Loss: ['0.0579', '0.0449', '0.0455', '0.4066']
Epoch [11/30], Loss: ['0.0625', '0.0497', '0.0544', '0.4451']
Epoch [12/30], Loss: ['0.0578', '0.0451', '0.0456', '0.2523']
Epoch [13/30], Loss: ['0.0637', '0.0515', '0.0557', '1.0345']
Epoch [14/30], Loss: ['0.0570', '0.0498', '0.0561', '0.6796']
Epoch [15/30], Loss: ['0.0587', '0.0481', '0.0487', '0.4368']
Epoch [16/30], Loss: ['0.0653', '0.0482', '0.0543',

[I 2024-11-15 01:04:42,645] Trial 1 finished with value: 0.4003 and parameters: {'output_size_0': 735, 'output_size_1': 663, 'output_size_2': 943, 'd_f_0': 0.8153903692088532, 'd_f_1': 0.009028437027154279, 'd_f_2': 0.11333105433957158, 'alpha_0': 1.8123843235799697, 'alpha_1': 0.7579446857438504, 'alpha_2': 1.232740717658952}. Best is trial 1 with value: 0.4003.


torch.Size([64, 1, 28, 28])
Epoch [1/30], Loss: ['0.6886', '0.8162', '1.1380', '5.8990']
Epoch [2/30], Loss: ['0.6526', '0.8281', '1.1749', '2.5865']
Epoch [3/30], Loss: ['0.7182', '0.7059', '1.1428', '2.2419']
Epoch [4/30], Loss: ['0.6571', '0.8937', '1.0580', '2.0924']
Epoch [5/30], Loss: ['0.6448', '0.8119', '0.9811', '2.7198']
Epoch [6/30], Loss: ['0.6748', '0.8597', '1.0433', '3.0347']
Epoch [7/30], Loss: ['0.6341', '0.8165', '1.0602', '2.0728']
Epoch [8/30], Loss: ['0.6617', '0.8746', '1.1070', '2.4560']
Epoch [9/30], Loss: ['0.6454', '0.7806', '0.9012', '1.7785']
Epoch [10/30], Loss: ['0.6700', '0.8484', '1.0350', '2.2765']
Epoch [11/30], Loss: ['0.6466', '0.8737', '1.0527', '2.2097']
Epoch [12/30], Loss: ['0.6428', '0.8618', '0.9749', '2.0495']
Epoch [13/30], Loss: ['0.6328', '0.8559', '1.0129', '1.8174']
Epoch [14/30], Loss: ['0.6326', '0.8191', '0.9275', '2.2583']
Epoch [15/30], Loss: ['0.6384', '0.8182', '0.9841', '2.2664']
Epoch [16/30], Loss: ['0.6431', '0.8433', '0.9524',

[I 2024-11-15 01:04:52,566] Trial 2 finished with value: 0.7289 and parameters: {'output_size_0': 1715, 'output_size_1': 935, 'output_size_2': 1533, 'd_f_0': 0.10653502124273984, 'd_f_1': 0.4281337758966294, 'd_f_2': 0.9236820373961723, 'alpha_0': 1.3621101624905176, 'alpha_1': 1.0496017411257144, 'alpha_2': 1.4846732647892475}. Best is trial 1 with value: 0.4003.


Best parameters: {'output_size_0': 735, 'output_size_1': 663, 'output_size_2': 943, 'd_f_0': 0.8153903692088532, 'd_f_1': 0.009028437027154279, 'd_f_2': 0.11333105433957158, 'alpha_0': 1.8123843235799697, 'alpha_1': 0.7579446857438504, 'alpha_2': 1.232740717658952}
Best validation loss: 0.4003
